In [1]:
import pandas as pd
df = pd.read_csv("titanic.csv",index_col = 0)
df.head()

,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [2]:
df.drop(['Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
PassengerId,,,,,
1,3,male,22.0,7.2500,0
2,1,female,38.0,71.2833,1
3,3,female,26.0,7.9250,1
4,1,female,35.0,53.1000,1
5,3,male,35.0,8.0500,0


In [3]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [4]:
dummies = pd.get_dummies(inputs.Sex ,prefix='Sex',columns=['male','female'], drop_first=True, dtype=int)
dummies.head()

,Sex_male
PassengerId,
1,1
2,0
3,0
4,0
5,1


In [5]:
# inputs.Sex = inputs.Sex.map({'male': 0, 'female': 1})

In [6]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,Sex_male
PassengerId,,,,,
1,3,male,22.0,7.2500,1
2,1,female,38.0,71.2833,0
3,3,female,26.0,7.9250,0


In [7]:
inputs.drop('Sex',axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,Sex_male
PassengerId,,,,
1,3,22.0,7.2500,1
2,1,38.0,71.2833,0
3,3,26.0,7.9250,0


In [8]:
inputs.isna().any()

Pclass      False
Age          True
Fare        False
Sex_male    False
dtype: bool

In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3,random_state=2023)

In [12]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB().fit(X_train,y_train)
model.score(X_test,y_test)

0.7611940298507462

In [13]:
model.predict(X_test[0:10]), y_test[0:10]

(array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1], dtype=int64),
 PassengerId
 841    0
 555    1
 215    0
 440    0
 367    1
 436    1
 546    0
 263    0
 351    0
 256    1
 Name: Survived, dtype: int64)

In [14]:
model.predict_proba(X_test[:10])

array([[9.68505479e-01, 3.14945211e-02],
       [4.19594200e-01, 5.80405800e-01],
       [9.72626292e-01, 2.73737080e-02],
       [9.40421411e-01, 5.95785894e-02],
       [1.18366517e-02, 9.88163348e-01],
       [4.69157246e-04, 9.99530843e-01],
       [6.86258582e-01, 3.13741418e-01],
       [3.42059332e-01, 6.57940668e-01],
       [9.70495455e-01, 2.95045449e-02],
       [4.50638899e-01, 5.49361101e-01]])

**Calculate the score using cross validation**

In [15]:
import numpy as np
from sklearn.model_selection import cross_val_score
model = cross_val_score(GaussianNB(),X_train, y_train, cv=5)
np.mean(model)

0.7786193548387097